# Import packages

In [1]:
import os
import cmaps
import cmaps as cm
import cmocean
import numpy as np
import xarray as xr
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
from netCDF4 import Dataset
from datetime import date
from mpl_toolkits.basemap import Basemap
from matplotlib.pyplot import Polygon
from matplotlib import rcParams
from xarrayMannKendall import Mann_Kendall_test
from dask.distributed import Client
from astropy import convolution
rcParams['font.family'] = 'sans-serif'
directory   = '/g/data/fu5/eac/20year_run_Junde/jl4424/KC_GS'

In [ ]:
dataset0  = sio.loadmat(directory+'/Figures/Figure1_trend.mat')
dataset1  = sio.loadmat(directory+'/Figures/Figure2_mean.mat')
dataset2  = xr.open_dataset(directory+'/ERA5/ERA5_monthly_Tx_LS.nc')
dataset3  = xr.open_dataset(directory+'/ERA5/ERA5_monthly_u10_LS.nc')
dataset4  = xr.open_dataset(directory+'/AVISO/AVISO_monthly_SSH_LS_detrend.nc')
sst_trend = dataset0['sst_trend'][:,:-1]*3650
sst_mean  = dataset1['oisst_mean'][:,:-1]
lon       = np.tile(dataset2.lon, (len(dataset2.lat),1)).transpose()
lat       = np.tile(dataset2.lat, (len(dataset2.lon),1))
u10       = np.nanmean(dataset3.u10.values,axis=0).transpose()
u10_mean  = np.fliplr(np.concatenate((u10[1199:,160:361],u10[:681,160:361]),axis=0))
u10_mean[np.where(np.isnan(sst_mean))]=np.nan
u10_mean[np.where((lon>330) & (lat<=30))]=np.nan
u10_mean[np.where(lat<10)]=np.nan
large_scale_ssh1 = dataset4.adt.transpose().values
large_scale_ssh2 = large_scale_ssh1.copy()
large_scale_ssh1[:,:30,:]=np.nan;
large_scale_ssh1[:,182:,:]=np.nan;
# large_scale_ssh1[:5,:,:]=np.nan;
large_scale_ssh2[:,:30,:]=np.nan;
large_scale_ssh2[:636,:,:]=np.nan;
large_scale_ssh2[905:,:,:]=np.nan;
KC_SSH = np.nanmean(large_scale_ssh1[:,1:,:],axis=2)
GS_SSH = np.nanmean(large_scale_ssh2[:,1:,:],axis=2)
KC_SSH_SG = KC_SSH.copy()
GS_SSH_SG = GS_SSH.copy()
KC_SSH[np.where((lon>173) | (lat<=21))]=np.nan
GS_SSH[np.where(lon>313)]=np.nan

# Plot the Northern Hemisphere

In [ ]:
fig_ratio=0.9
fig = plt.figure(figsize=(30, 20))
gs=gridspec.GridSpec(3,3)
labelfont=18
padspacescale=10
labelpadscale=2
linefont = 6
scale    = 1.0
levels1  = np.linspace(-1,1,100)
tick_marks1 = np.array([-1.0,-0.5,0,0.5,1.0])
# cmaps1  = cmaps.cmocean_balance
cmaps1  = cm.BlueWhiteOrangeRed
##################################################################################
ax = fig.add_subplot(gs[0:2])
l, b, w, h = ax.get_position().bounds
m   = Basemap(projection='merc',llcrnrlat=np.nanmin(lat),urcrnrlat= np.nanmax(lat),llcrnrlon=np.nanmin(lon),urcrnrlon=np.nanmax(lon),resolution='l')    
m.drawmeridians(np.arange(0,360, 20),labels=[0,0,0,1],linewidth=0.3*linefont,dashes=[2,2],color='.5',fontsize=1.2*labelfont,family='sans-serif')
m.drawparallels(np.arange(0,55, 10),labels=[1,0,0,0],linewidth=0.3*linefont,dashes=[2,2],color='.5',fontsize=1.2*labelfont,family='sans-serif')  
x, y = m(lon, lat)
CB=m.contourf(x, y, sst_trend,cmap=cmaps1,levels=levels1,origin='lower',extend='both')     
CS=m.contour(x, y, u10_mean,levels=[0.0],   linewidths=0.8*linefont,linestyles='solid', colors='xkcd:cherry red')
CS=m.contour(x, y, KC_SSH_SG, levels=[1.1], linewidths=0.5*linefont,linestyles='solid',colors='tab:gray')
CS=m.contour(x, y, GS_SSH_SG, levels=[0.25],linewidths=0.5*linefont,linestyles='solid',colors='tab:blue') 
CS=m.contour(x, y, KC_SSH,    levels=[1.1], linewidths=1.0*linefont,linestyles='solid',colors='tab:gray')
CS=m.contour(x, y, GS_SSH,    levels=[0.25],linewidths=1.0*linefont,linestyles='solid',colors='tab:blue')    
ax.set_position([l, b, scale*w, scale*h])
ax.spines['left'].set_linewidth(linefont)
ax.spines['top'].set_linewidth(linefont)
ax.spines['right'].set_linewidth(linefont)
ax.spines['bottom'].set_linewidth(linefont)
plt.xlabel('Longitude',fontsize=1.2*labelfont,labelpad=4*padspacescale,family='sans-serif')
plt.ylabel('Latitude',fontsize=1.2*labelfont,labelpad=7*padspacescale,family='sans-serif')
ax.axes.xaxis.set_tick_params(pad=0)
ax.axes.yaxis.set_tick_params(pad=0)
m.drawcoastlines(color='0.1',  linewidth=0.2*linefont)
m.drawmapboundary(color='0.1', linewidth=0.2*linefont)
m.fillcontinents(color='0.9', lake_color='white')
cbaxes1     = fig.add_axes([0.63, 0.675, 0.008, 0.185])
cb1 = plt.colorbar(CB,orientation='vertical',cax = cbaxes1)
cb1.set_ticks(tick_marks1)
cb1.ax.tick_params(labelsize=labelfont)
cb1.set_label(r'SST trend ($^\circ$C per decade)', fontsize=labelfont,labelpad=0,family='sans-serif')
fig.savefig(directory+'/Figures/Figure6_Schematic.png',dpi=300,bbox_inches = 'tight')